# Lab 3 - Spark MLlib

"A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E"
-Tom M. Mitchell

Machine Learning - the science of getting computers to act without being explicitly programmed

MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. It consists of common learning algorithms and utilities, including classification, regression, clustering, collaborative filtering (this example!), dimensionality reduction, as well as lower-level optimization primitives and higher-level pipeline APIs.

It divides into two packages:
1. spark.mllib contains the original API built on top of RDDs.
2. spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.


Using spark.ml is recommended because with DataFrames the API is more versatile and flexible. But we will keep supporting spark.mllib along with the development of spark.ml. Users should be comfortable using spark.mllib features and expect more features coming.

http://spark.apache.org/docs/latest/mllib-guide.html

## Online Purchase Recommendations

Learn how to create a recommendation engine using the Alternating Least Squares algorithm in Spark's machine learning library

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/ALS.png' width="70%" height="70%"></img>

## The data

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.  The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

http://archive.ics.uci.edu/ml/datasets/Online+Retail

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/FullFile.png' width="80%" height="80%"></img>

## Step 1 - Create an RDD from the CSV File 
### 1.1 - Download the data

In [1]:
#Download the data from github to the local directory
!rm 'OnlineRetail.csv.gz' -f
!wget https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz

--2016-10-17 17:01:52--  https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/OnlineRetail.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7483128 (7.1M) [application/octet-stream]
Saving to: 'OnlineRetail.csv.gz'

100%[======================================>] 7,483,128   --.-K/s   in 0.1s    

2016-10-17 17:01:54 (69.4 MB/s) - 'OnlineRetail.csv.gz' saved [7483128/7483128]



### 1.2 - Put the csv into an RDD (at first, each row in the RDD is a string which correlates to a line in the csv) and show the first three lines.
<br>
 <div class="panel-group" id="accordion-12">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-1" href="#collapse1-12">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-12" class="panel-collapse collapse">
      <div class="panel-body">Use the Spark context (sc) to get the list of possible methods.  <i>sc.&lt;TAB&gt;</i></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-12" href="#collapse2-12">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-12" class="panel-collapse collapse">
      <div class="panel-body">Use the <i>textFile()</i> method</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-12" href="#collapse3-12">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-12" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
loadRetailData = sc.textFile("OnlineRetail.csv.gz")<br>
loadRetailData.take(3)<br></div>
    </div>
  </div>
</div> 



In [2]:
loadRetailData = sc.textFile("OnlineRetail.csv.gz")
loadRetailData.take(3)

[u'InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country',
 u'536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850,United Kingdom',
 u'536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850,United Kingdom']

## Step 2 - Prepare and shape the data:  "80% of a Data Scientists  job"

### 2.1 - Remove the header from the RDD and split the remaining lines by comma.
<br>
 <div class="panel-group" id="accordion-21">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-21" href="#collapse1-21">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-21" class="panel-collapse collapse">
      <div class="panel-body">The header is the first line in the RDD -- use <i>first()</i> to obtain it.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-21" href="#collapse2-21">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-21" class="panel-collapse collapse">
      <div class="panel-body">Use the <i>filter()</i> method to filter out all lines which are not equal to the header line.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-21" href="#collapse3-21">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-21" class="panel-collapse collapse">
      <div class="panel-body">Map the <i>split()</i> method to the remaining lines to split on ","</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-21" href="#collapse4-21">
        Hint 4</a>
      </h4>
    </div>
    <div id="collapse4-21" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>

header = loadRetailData.first()<br>
splitColumns = loadRetailData.filter(lambda line: line != header).map(lambda l: l.split(","))</div>
    </div>
  </div>
</div> 

In [3]:
header = loadRetailData.first()
splitColumns = loadRetailData.filter(lambda line: line != header).map(lambda l: l.split(","))

### 2.2 - Filter the remaining lines using <a href="https://docs.python.org/2.6/howto/regex.html">regular expressions</a>
The original file at UCI's Machine Learning Repository has commas in the product description.  Those have been removed to expediate the lab.
Only keep rows that have a quantity greater than 0, a non-empty customerID, and a non-blank stock code after removing non-numeric characters.<br><br>
 <div class="panel-group" id="accordion-22">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse1-22">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-22" class="panel-collapse collapse">
      <div class="panel-body">Examine the header to determine which fields need to be used to filter the data.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse2-22">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-22" class="panel-collapse collapse">
      <div class="panel-body">Use the <i>filter()</i> method for the first two requirements.   Note -- you may have to cast values.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse3-22">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-22" class="panel-collapse collapse">
      <div class="panel-body">Look at the <i><a href="https://docs.python.org/2.6/howto/regex.html">re.sub()</a></i> method</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-22" href="#collapse4-22">
        Hint 4</a>
      </h4>
    </div>
    <div id="collapse4-22" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
import re<br>
filteredRetailData = splitColumns.filter(lambda l: int(l[3]) > 0 and len(re.sub("\D", "", l[1])) != 0 and l[6] != "")</div>
    </div>
  </div>
</div> 

In [4]:
import re
filteredRetailData = splitColumns.filter(lambda l: int(l[3]) > 0 and len(re.sub("\D", "", l[1])) != 0 and l[6] != "")

###  2.3 - Map each line to a SQL Row and create a Dataframe from the result.   Register the Dataframe as an SQL temp table.
<br>
Use the following for the Row column names: inv, stockCode, description, quant, invDate, price, custId, country.   inv, stockCode, quant and custId should be integers.   
price is a float.  description and country are strings (the default).
<br><br>
Hint: When you replaced non-digit characters using the regular expression above, you replaced them in the context of a test.  You'll have to do it again when creating the stockCode Row value.
<br><br>
 <div class="panel-group" id="accordion-23">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-23" href="#collapse1-23">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-23" class="panel-collapse collapse">
      <div class="panel-body">We haven't used SQLContext or Row in this notebook, so you will have to import them from the pyspark.sql package and then create a <i>SQLContext</i>.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-23" href="#collapse2-23">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-23" class="panel-collapse collapse">
      <div class="panel-body">You can create a <i>Row</i> using a <i>map()</i>.   For example:<br>
      example = myRDD.map(lambda x: Row(v1=x[1], v2=int(x[2]), v3=float(x[3]))<br>
      Note how we set the column names this way.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-23" href="#collapse3-23">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-23" class="panel-collapse collapse">
      <div class="panel-body">use <i>createDataFrame()</i> in your <i>SQLContext</i>.   Then register the dataframe with <i>registerTempTable()</i></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-23" href="#collapse4-23">
        Hint 4</a>
      </h4>
    </div>
    <div id="collapse4-23" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
from pyspark.sql import SQLContext, Row<br>
sqlContext = SQLContext(sc)<br>

retailRows = filteredRetailData.map(lambda l: Row(inv=int(l[0]), stockCode=int(re.sub("\D", "", l[1])), description=l[2], quant=int(l[3]), invDate=l[4], price=float(l[5]), custId=int(l[6]), country=l[7]))<br>

retailDf = sqlContext.createDataFrame(retailRows)<br>
retailDf.registerTempTable("retailPurchases")</div>
    </div>
  </div>
</div> 

In [5]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)
retailRows = filteredRetailData.map(lambda l: Row(inv=int(l[0]), stockCode=int(re.sub("\D", "", l[1])), description=l[2], quant=int(l[3]), invDate=l[4], price=float(l[5]), custId=int(l[6]), country=l[7]))
retailDf = sqlContext.createDataFrame(retailRows)
retailDf.registerTempTable("retailPurchases")
values = sqlContext.sql("select * from retailPurchases where stockCode='84632'")
values.toPandas()

,country,custId,description,inv,invDate,price,quant,stockCode
0,United Kingdom,13901,DECORATIVE HANGING SHELVING UNIT,541990,1/25/11 9:01,59.95,1,84632
1,United Kingdom,16638,DECORATIVE HANGING SHELVING UNIT,545704,3/7/11 8:30,59.95,2,84632
2,United Kingdom,15696,DECORATIVE HANGING SHELVING UNIT,548038,3/29/11 12:03,59.95,1,84632
3,United Kingdom,17663,DECORATIVE HANGING SHELVING UNIT,559420,7/8/11 11:52,59.95,1,84632
4,United Kingdom,17175,DECORATIVE HANGING SHELVING UNIT,560914,7/21/11 18:08,59.95,1,84632


### 2.4 - Keep only the data we need (custId, stockCode, and rank)
<br>
The Alternating Least Squares algorithm requires three values.   In this case, we're going to use the Customer ID (custId), stock code (stockCode) and a ranking value.   In this situation there is not a ranking value within the data, so we will create one.   We will set a value of 1 to indicate a purchase since these are all actual orders.   Set that value to "purch".
<br><br>
After doing the select, group by custId and stockCode.
<br><br>
 <div class="panel-group" id="accordion-24">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-24" href="#collapse1-24">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-24" class="panel-collapse collapse">
      <div class="panel-body">To add a fixed value within a select statement, use something like <i>select x,y,1 as purch from z</i></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-24" href="#collapse2-24">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-24" class="panel-collapse collapse">
      <div class="panel-body">Use the <i>group by</i> statement to group results.  To group by two values, separate them by commas (i.e. <i>group by x,y</i>)</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-24" href="#collapse3-24">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-24" class="panel-collapse collapse">
      <div class="panel-body">Type:

query = "
SELECT 
    custId, stockCode, 1 as purch
FROM 
    retailPurchases 
group 
    by custId, stockCode"<br>
uniqueCombDf = sqlContext.sql(query)</div>
    </div>
  </div>
</div> 

In [6]:
query = " SELECT custId, stockCode, 1 as purch FROM retailPurchases group by custId, stockCode"
uniqueCombDf = sqlContext.sql(query)

### 2.5 - Randomly split the data into a testing set (10% of the data), a cross validation set (10% of the data) a training set (80% of the data)
<br><br>
We wish to split up the data into three parts.   A training set (80%) to train the algorithm, a testing set (10%) and a cross-validation set (10%).   The data for each set should be randomly selected.
<br><br>
 <div class="panel-group" id="accordion-25">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-25" href="#collapse1-25">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-25" class="panel-collapse collapse">
      <div class="panel-body">Use the <i><a href="https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit">randomSplit()</a></i> method</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-25" href="#collapse2-25">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-25" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
      testDf, cvDf, trainDf = uniqueCombDf.randomSplit([.1,.1,.8])<br></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-25" href="#collapse3-25">
        Advanced Optional 1</a>
      </h4>
    </div>
    <div id="collapse3-25" class="panel-collapse collapse">
      <div class="panel-body"><i>randomSplit()</i> takes an optional seed parameter.  At the end of the exercise give a random seed and see whether the results change.</div>
    </div>
  </div>
</div> 

In [7]:
testDf, cvDf, trainDf = uniqueCombDf.randomSplit([.1,.1,.8])

## Step 3 - Build recommendation models

### 3.1 - Use the training dataframe to train a model with Alternating Least Squares using the <i><a href="https://spark.apache.org/docs/1.6.1/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS">ALS</a></i> class
<br>
ALS attempts to estimate the ratings matrix R as the product of two lower-rank matrices, X and Y, i.e. X * Yt = R. Typically these approximations are called ‘factor’ matrices. The general approach is iterative. During each iteration, one of the factor matrices is held constant, while the other is solved for using least squares. The newly-solved factor matrix is then held constant while solving for the other factor matrix.
<br><br>
Latent Factors / rank<br>
&nbsp;&nbsp;&nbsp;&nbsp;The number of columns in the user-feature and product-feature matricies<br>
Iterations / maxIter<br>
&nbsp;&nbsp;&nbsp;&nbsp;The number of factorization runs<br><br>
To use the ALS class type:
<br>
from pyspark.ml.recommendation import ALS<br>
<br>
When running ALS, we need to create two separate instances.   For both instances userCol is custId, itemCol is stockCode and ratingCol is purch.<br><br>
For the first instance, use a rank of 15 and set iterations to 5.<br>
For the second instance, use a rank of 2 and set iterations to 10.<br>
Run <i><a href="https://spark.apache.org/docs/1.6.1/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS.fit">fit()</a></i> on both instances using the training dataframe.<br>
<br>
 <div class="panel-group" id="accordion-31">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-31" href="#collapse1-31">
        Advanced Optional 1</a>
      </h4>
    </div>
    <div id="collapse1-31" class="panel-collapse collapse">
      <div class="panel-body">Create an emply instance of the <i>ALS</i> class and run the <i>explainParams</i> method on it to see the default values.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-31" href="#collapse2-31">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse2-31" class="panel-collapse collapse">
      <div class="panel-body">als1 = ALS(rank=15, maxIter=5, userCol="custId", itemCol="stockCode", ratingCol="purch")</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-31" href="#collapse3-31">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse3-31" class="panel-collapse collapse">
      <div class="panel-body">model1 = als1.fit(trainDf)</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-31" href="#collapse4-31">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse4-31" class="panel-collapse collapse">
      <div class="panel-body">Type:
<br>
from pyspark.ml.recommendation import ALS<br>

als1 = ALS(rank=15, maxIter=5, userCol="custId", itemCol="stockCode", ratingCol="purch")<br>
model1 = als1.fit(trainDf)<br>

als2 = ALS(rank=2, maxIter=10, userCol="custId", itemCol="stockCode", ratingCol="purch")<br>
model2 = als2.fit(trainDf)
</div>
    </div>
  </div>
</div> 


In [8]:
from pyspark.ml.recommendation import ALS

als1 = ALS(rank=15, maxIter=5, userCol="custId", itemCol="stockCode", ratingCol="purch")
model1 = als1.fit(trainDf)

als2 = ALS(rank=2, maxIter=10, userCol="custId", itemCol="stockCode", ratingCol="purch")
model2 = als2.fit(trainDf)
als3 = ALS()
als3.explainParams()

'alpha: alpha for implicit preference (default: 1.0)\ncheckpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. (default: 10)\nimplicitPrefs: whether to use implicit preference (default: False)\nitemCol: column name for item ids (default: item)\nmaxIter: max number of iterations (>= 0). (default: 10)\nnonnegative: whether to use nonnegative constraint for least squares (default: False)\nnumItemBlocks: number of item blocks (default: 10)\nnumUserBlocks: number of user blocks (default: 10)\npredictionCol: prediction column name. (default: prediction)\nrank: rank of the factorization (default: 10)\nratingCol: column name for ratings (default: rating)\nregParam: regularization parameter (>= 0). (default: 0.1)\nseed: random seed. (default: None)\nuserCol: column name for user ids (default: user)'

## Step 4 - Test the models

Use the models to predict what the user will rate a certain item.  The closer our model is to 1 for an item a user has already purchased, the better.

### 4.1 - Evaluate the model with the cross validation dataframe by using the transorm function.

Some of the users or purchases in the cross validation data may not have been in the training data.  Let's remove the ones that aren't.   To do this obtain all the the custId and stockCode values from the training data and filter out any lines with those values from the cross-validation data.
<br><br>
At the end, print out how many cross-validation lines we had at the start -- and the new number afterwords.
<br><br>
 <div class="panel-group" id="accordion-41">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-41" href="#collapse1-41">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-41" class="panel-collapse collapse">
      <div class="panel-body">Use <i>map()</i> to return a specific value (i.e. foo = foo.map(lambda x: x.value)) and put them all in a set (i.e. foo1 = set(foo))<br></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-41" href="#collapse2-41">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-41" class="panel-collapse collapse">
      <div class="panel-body">You need all the returned values (remember they might be spread all across the cluster!) so run collect() on the results of the map(). (i.e. foo1 = set(foo.collect()))</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-41" href="#collapse3-41">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-41" class="panel-collapse collapse">
      <div class="panel-body">Use the <i>filter()</i> to filter out any values in the cross-validation dataframe which are in the stockCode or custId sets.   Use <i>toDF()</i> to change the results to a dataframe.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-41" href="#collapse4-41">
        Hint 4</a>
      </h4>
    </div>
    <div id="collapse4-41" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
customers = set(trainDf.rdd.map(lambda line: line.custId).collect())<br>
stock = set(trainDf.rdd.map(lambda line: line.stockCode).collect())<br>

filteredCvDf = cvDf.rdd.filter(lambda line: line.stockCode in stock and line.custId in customers).toDF()<br>

print cvDf.count()<br>
print filteredCvDf.count()<br></div>
    </div>
  </div>
</div> 

In [9]:
customers = set(trainDf.rdd.map(lambda line: line.custId).collect())
stock = set(trainDf.rdd.map(lambda line: line.stockCode).collect())

filteredCvDf = cvDf.rdd.filter(lambda line: line.stockCode in stock and line.custId in customers).toDF()

print cvDf.count()
print filteredCvDf.count()

25762
25733


### Step 4.2 - Make Predictions using <i><a href="https://spark.apache.org/docs/1.6.1/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALSModel.transform">transform()</a></i>

Type:

predictions1 = model1.transform(filteredCvDf)<br>
predictions2 = model2.transform(filteredCvDf)
</font>

In [10]:
predictions1 = model1.transform(filteredCvDf)
predictions2 = model2.transform(filteredCvDf)

### 4.3 - Calculate and print the Mean Squared Error.   For all ratings, subtract the prediction from the actual purchase (1), square the result, and take the mean of all of the squared differences.

The lower the result number, the better the model.

Type:

meanSquaredError1 = predictions1.map(lambda line: (line.purch - line.prediction)\*\*2).mean()<br>
meanSquaredError2 = predictions2.map(lambda line: (line.purch - line.prediction)\*\*2).mean()<br><br>
    
print 'Mean squared error = %.4f for our first model' % meanSquaredError1<br>
print 'Mean squared error = %.4f for our second model' % meanSquaredError2


In [11]:
meanSquaredError1 = predictions1.map(lambda line: (line.purch - line.prediction)**2).mean()
meanSquaredError2 = predictions2.map(lambda line: (line.purch - line.prediction)**2).mean()

print 'Mean squared error = %.4f for our first model' % meanSquaredError1
print 'Mean squared error = %.4f for our second model' % meanSquaredError2

Mean squared error = 0.0116 for our first model
Mean squared error = 0.0098 for our second model


### 4.4 - Confirm the model by testing it with the test data and the best hyperparameters found during cross-validation

Filter the test dataframe (testDf) the same way as the cross-validation dataframe.   Then run the transform() and calculate the mean squared error.   It should be the same as the value calcuated above.

Type:

filteredTestDf = testDf.rdd.filter(lambda line: line.stockCode in stock and line.custId in customers).toDF()<br>
predictions3 = model2.transform(filteredTestDf)<br>
meanSquaredError3 = predictions3.map(lambda line: (line.purch - line.prediction)\*\*2).mean()<br><br>
    
print 'Mean squared error = %.4f for our best model' % meanSquaredError3


In [12]:
filteredTestDf = testDf.rdd.filter(lambda line: line.stockCode in stock and line.custId in customers).toDF()
predictions3 = model2.transform(filteredTestDf)
meanSquaredError3 = predictions3.map(lambda line: (line.purch - line.prediction)**2).mean()

print 'Mean squared error = %.4f for our best model' % meanSquaredError3

Mean squared error = 0.0098 for our best model


## Step 5 - Implement the model

### 5.1 - First, create a dataframe in which each row has the user id and an item id.
<br>
Use the <i><a href="https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame">Dataframe</a></i> methods to create a Dataframe with a specific user and that user's purchased products.<br>
&nbsp;&nbsp;&nbsp;&nbsp;First, use the Dataframe <i><a href="https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.filter">filter()</a></i> to filter out all custId's but 15544.<br>
&nbsp;&nbsp;&nbsp;&nbsp;Then use the <i><a href="https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select">select()</a></i> to only return the <i>custId</i> column.<br>
&nbsp;&nbsp;&nbsp;&nbsp;Now use <i><a href="https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct">distinct()</a></i> to ensure we only have the single custId.<br>
&nbsp;&nbsp;&nbsp;&nbsp;Do a <i><a href="https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">join()</a></i> with the distinct values from the stockCode column.
<br><br>
 <div class="panel-group" id="accordion-51">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-51" href="#collapse1-51">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-51" class="panel-collapse collapse">
      <div class="panel-body">Use the Dataframe <i>filter()</i> method to filter out all users but 15544<br>
      user = trainDf.filter(trainDf.custId == 15544)<br></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-51" href="#collapse2-51">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-51" class="panel-collapse collapse">
      <div class="panel-body">Use the Dataframe <i>select()</i> method to only select the custId column<br>
      userCustId = user.select("custId")</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-51" href="#collapse3-51">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-51" class="panel-collapse collapse">
      <div class="panel-body">Use the Dataframe <i>distinct()</i> method to only return unique rows.<br>
      userCustIdDistinct = userCustId.distinct()</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-51" href="#collapse4-51">
        Hint 4</a>
      </h4>
    </div>
    <div id="collapse4-51" class="panel-collapse collapse">
      <div class="panel-body">Use the Dataframe <i>join()</i> method to join the results with distinct stockCodes</div>
    </div>
  </div>
    <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-51" href="#collapse4-51">
        Hint 5</a>
      </h4>
    </div>
    <div id="collapse5-51" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
user = trainDf.filter(trainDf.custId == 15544)<br>
userCustId = user.select("custId")<br>
userCustIdDistinct = userCustId.distinct()<br>
stockCode = trainDf.select("stockCode")<br>
stockCodeDistinct = stockCode.distinct()<br>
userItems = userCustIdDistinct.join(stockCodeDistinct)<br>
OR
userItems = trainDf.filter(trainDf.custId == 15544).select("custId").distinct().join( trainDf.select("stockCode").distinct())<br></div>
    </div>
  </div>
</div> 

In [13]:
userItems = trainDf.filter(trainDf.custId == 15544).select("custId").distinct().join( trainDf.select("stockCode").distinct())
#user = trainDf.filter(trainDf.custId == 15544)
#userCustId = user.select("custId")
#userCustIdDistinct = userCustId.distinct()
#stockCode = trainDf.select("stockCode")
#stockCodeDistinct = stockCode.distinct()
#userItems = userCustIdDistinct.join(stockCodeDistinct)




### 5.2 - Use 'transform' to rate each item.

Type:

bestRecsDf = model2.transform(userItems)
bestRecsDf.first()


In [16]:
bestRecsDf = model2.transform(userItems)
bestRecsDf.first()

Row(custId=15544, stockCode=20831, prediction=0.9010576009750366)

###  5.3 - Print the top 5 recommendations sorted on prediction.

 <div class="panel-group" id="accordion-53">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-53" href="#collapse1-53">
        Hint 1</a>
      </h4>
    </div>
    <div id="collapse1-53" class="panel-collapse collapse">
      <div class="panel-body">In order to print the top five recommendations, we need to <i><a href"https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sort">sort()</a></i> them in descending order<br></div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-53" href="#collapse2-53">
        Hint 2</a>
      </h4>
    </div>
    <div id="collapse2-53" class="panel-collapse collapse">
      <div class="panel-body">Use <i>take()</i> to get the top 5 values.</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-53" href="#collapse3-53">
        Hint 3</a>
      </h4>
    </div>
    <div id="collapse3-53" class="panel-collapse collapse">
      <div class="panel-body">Type:<br>
      print bestRecsDf.sort("prediction",ascending=False).take(5)</div>
    </div>
  </div>
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" data-parent="#accordion-53" href="#collapse4-53">
        Advanced Optional 1</a>
      </h4>
    </div>
    <div id="collapse4-53" class="panel-collapse collapse">
      <div class="panel-body">select from the retailPurchases temp table on stockCode to see some of selections recommended.</div>
    </div>
  </div>
</div> 

In [15]:
print bestRecsDf.sort("prediction",ascending=False).take(5)
values = sqlContext.sql("select * from retailPurchases where stockCode='35610'")
values.toPandas()

[Row(custId=15544, stockCode=90012, prediction=0.901187002658844), Row(custId=15544, stockCode=35400, prediction=0.9010587334632874), Row(custId=15544, stockCode=84857, prediction=0.901058554649353), Row(custId=15544, stockCode=90000, prediction=0.901058554649353), Row(custId=15544, stockCode=47556, prediction=0.9010583162307739)]


,country,custId,description,inv,invDate,price,quant,stockCode
0,United Kingdom,15361,BLACK CHRISTMAS FLOCK DROPLET,538527,12/13/10 9:52,1.25,24,35610
1,United Kingdom,15361,WHITE CHRISTMAS FLOCK DROPLET,538527,12/13/10 9:52,1.25,24,35610
2,United Kingdom,15512,PINK CHRISTMAS FLOCK DROPLET,538993,12/15/10 12:01,1.25,8,35610


Let's look up this user and the recommended product ID's in the excel file...

<img src='https://raw.githubusercontent.com/rosswlewis/RecommendationPoT/master/user.png' width="80%" height="80%"></img>

This user seems to have purchased a lot of childrens gifts and some holiday items.  The recommendation engine we created suggested some items along these lines


#####  Citation
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).